In [11]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import transformers

# cfg

In [4]:
class CFG:
    model_test=True
    frac=0.01#0.02 #1.0#
    pretrained=True #False#
    model= "tf_efficientnet_b0"#"resnet18"#"tf_efficientnet_b0"#"resnet18"
    train_bs=4
    valid_bs=4#12
    test_bs=4
    epoch=2
    fold=[0]
    seed= 2022
    lr=2e-4
    es=4
    device="cuda"

    # make fold
    make_fold=True
    
    #train, inference
    _train=False#True
    _predict=True

    #file path
    path_train="E:/kaggle_data/Jigsaw Rate Severity of Toxic Comments/jigsaw-toxic-comment-classification-challen/train.csv"
    path_test="E:/kaggle_data/Jigsaw Rate Severity of Toxic Comments/sample_submission.csv"
    path_test2="E:/kaggle_data/Jigsaw Rate Severity of Toxic Comments/comments_to_score.csv"
    path_ref_model="bert-base-uncased"
    
    # path_train="../input/jigsaw-toxic-comment-classification-challenge/train.csv"
    # path_test="../input/jigsaw-toxic-severity-rating/sample_submission.csv"
    # path_test2="../input/jigsaw-toxic-severity-rating/sample_submission.csv"
    # path_ref_model="../input/bert-base-uncased"

    #customize
    max_sens=512


# preprocess

In [5]:
def clean(data):

    # Clean some punctutations
    data = re.sub('\n', ' ', data)
    # Remove ip address
    data = re.sub(r'(([0-9]+\.){2,}[0-9]+)',' ', data)
    
    data = re.sub(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3', data)
    # Replace repeating characters more than 3 times to length of 3
    data = re.sub(r'([*!?\'])\1\1{2,}',r'\1\1\1', data)
    # patterns with repeating characters 
    data = re.sub(r'([a-zA-Z])\1{2,}\b',r'\1\1', data)
    data = re.sub(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1', data)

    # Add space around repeating characters
    data = re.sub(' +', ' ', data)
    
    # Ex) I didn ' t -> I didn't
    data = re.sub(" ' ", "'", data)
    
    return data

def get_data():
    train=pd.read_csv(CFG.path_train)
    train["target"]=(train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) > 0).astype(int)
    if CFG.make_fold:
        skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=5)
        train["fold"]=-1
        for i,(_,val_idx) in enumerate(skf.split(train.sample(frac=1.),train.target)):
            train.loc[val_idx,"fold"]=i 
    # train.comment_text=train.comment_text.apply(lambda x: clean(x))
    return train    

def get_data_test():
    comments_to_score = pd.read_csv(CFG.path_test2)
    comments_to_score.columns=["id","comment_text"]
    comments_to_score["target"]=-1
    return comments_to_score

# data loader

In [6]:
import random
import os
import numpy as np
from PIL import Image

from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import Dataset, DataLoader

from cfg import get_cfg
CFG=get_cfg()

import transformers
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-uncased")

def random_seed(SEED):
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
random_seed(CFG.seed)

class SetiDataset(Dataset):
    def __init__(self, df, transform=None, conf=None):
        self.df = df.reset_index(drop=True)
        self.labels = df['target'].values
        self.transform = transform
        self.conf = conf
        
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, idx):
        sentence = self.df.loc[idx, 'comment_text']

        bert_sens = tokenizer.encode_plus(
                                sentence,
                                add_special_tokens = True, 
                                max_length = CFG.max_sens,#2502, # 上で314に設定しています
                                pad_to_max_length = True, 
                                return_attention_mask = True)
        ids = torch.tensor(bert_sens['input_ids'], dtype=torch.long)
        mask = torch.tensor(bert_sens['attention_mask'], dtype=torch.long)
        token_type_ids = torch.tensor(bert_sens['token_type_ids'], dtype=torch.long)

        target = torch.tensor(self.labels[idx],dtype=torch.float)
        
        return {
            'ids': ids,
            'mask': mask,
            'token_type_ids': token_type_ids,
            'targets': target
        }

def get_dataset(train=None,fold=0):
    train_df=train[train.fold!=fold].reset_index(drop=True).sample(frac=CFG.frac)
    valid_df=train[train.fold==fold].reset_index(drop=True).sample(frac=CFG.frac)
    train_dataset=SetiDataset(train_df)
    valid_dataset=SetiDataset(valid_df)
    return train_dataset,valid_dataset

def get_dataloader(train=None,fold=0):
    train_dataset,valid_dataset = get_dataset(train,fold)
    train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=CFG.train_bs,num_workers=0,shuffle=True)
    valid_loader=torch.utils.data.DataLoader(valid_dataset,batch_size=CFG.valid_bs,num_workers=0,shuffle=False)
    return train_loader,valid_loader

def get_dataset_test(test=None,fold=0):
    test_dataset=SetiDataset(test.sample(frac=1.))
    return test_dataset

def get_dataloader_test(test=None,fold=0):
    test_dataset=get_dataset_test(test=test)
    test_loader=torch.utils.data.DataLoader(test_dataset,batch_size=CFG.test_bs,num_workers=0,shuffle=False)
    return test_loader


# model

In [7]:
from cfg import get_cfg
CFG=get_cfg()

import torch
import torch.nn as nn
import sys
sys.path.append(CFG.path_ref_model)
import transformers

def loss_fn(outputs,targets):
    # return nn.BCEWithLogitsLoss()(outputs,targets)
    return nn.BCEWithLogitsLoss()(outputs,targets)

def get_model(classes=1,pretrained=True):
    model = transformers.BertForSequenceClassification.from_pretrained(CFG.path_ref_model,num_labels=1)
    return model 

# train

In [8]:
from cfg import get_cfg
CFG=get_cfg()

import numpy as np
from model import get_model,loss_fn
import torch
import torch.nn as nn
from dataloader import get_dataset,get_dataloader
import os

## Engine
from tqdm import tqdm
class Engine:
    def __init__(self,model,optimizer,scheduler=None):
        self.model=model
        self.optimizer=optimizer
        self.scheduler=scheduler
        self.device=CFG.device

    def loss_fn(self,outputs,targets):
        return nn.BCEWithLogitsLoss()(outputs,targets)        

    def train(self,data_loader):
        self.model.train()
        final_loss=0
        for data in tqdm(data_loader):
            self.optimizer.zero_grad()
            ids = data["ids"].to(CFG.device,non_blocking=True)
            mask = data["mask"].to(CFG.device,non_blocking=True)
            outputs=self.model(ids,mask)
            outputs = outputs["logits"].squeeze(-1)
            targets = data["targets"].to(CFG.device,non_blocking=True)
            loss=self.loss_fn(outputs,targets)
            loss.backward()
            self.optimizer.step()
            final_loss += loss.item()
        return final_loss/len(data_loader)
    
    def validate(self,data_loader):
        self.model.eval()
        final_loss=0
        for data in tqdm(data_loader):
            with torch.no_grad():
                ids = data["ids"].to(CFG.device,non_blocking=True)
                mask = data["mask"].to(CFG.device,non_blocking=True)
                outputs=self.model(ids,mask)
                outputs = outputs["logits"].squeeze(-1)
                targets = data["targets"].to(CFG.device,non_blocking=True)
                loss=self.loss_fn(outputs,targets)
                final_loss += loss.item()
        return final_loss/len(data_loader)

def loop_train(train,fold=0,save_model=True):
    # Dataset
    train_loader,valid_loader=get_dataloader(train,fold)
    model=get_model()
    model.to(CFG.device)

    # model folder
    if os.path.isdir(CFG.model) == False:
        os.makedirs(CFG.model)

    # Model,Optimizer, scheduler, engine
    optimizer=torch.optim.Adam(model.parameters(),lr=3e-4)
    scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,patience=3,threshold=1e-5,mode="min",verbose=True)
    engine=Engine(model,optimizer,scheduler)
    best_loss=np.inf
    early_stopping=10
    early_stopping_cnt=0

    for epoch in range(CFG.epoch):
        train_loss=engine.train(train_loader)
        valid_loss=engine.validate(valid_loader)
        scheduler.step(valid_loss)

        if valid_loss<best_loss:
            best_loss=valid_loss
            torch.save(model.state_dict(),f"{CFG.model}/model_{CFG.model}_fold_{fold}.bin")
            print(f"fold={fold}, epoch={epoch}, train_loss={train_loss:.6f}, valid_loss={valid_loss:.6f}")    
            early_stopping_cnt=0
        else:
            early_stopping_cnt+=1
        if early_stopping_cnt>early_stopping:
            break

    print(f"fold={fold}, best val loss={best_loss}")        

# predict

In [9]:
from cfg import get_cfg
CFG=get_cfg()

import numpy as np
from model import get_model,loss_fn
import torch
import torch.nn as nn
from dataloader import get_dataset_test,get_dataloader_test
from tqdm import tqdm

def predict(model,dataloader):
    preds=[]
    with torch.no_grad():
        for data in tqdm(dataloader):
            ids = data["ids"].to(CFG.device,non_blocking=True)
            mask = data["mask"].to(CFG.device,non_blocking=True)
            outputs=model(ids,mask)
            outputs = outputs["logits"].squeeze(-1)
            preds.append(outputs.cpu().detach().numpy())
        preds=np.concatenate(preds)

    return preds

def loop_predict(test,fold=0,save_model=True):
    # Dataset
    test_loader=get_dataloader_test(test)

    all_preds=[]
    for pth in CFG.fold:
        model=get_model()
        model.to(CFG.device)
        model.eval()
        path=f"{CFG.model}/model_{CFG.model}_fold_{fold}.bin"
        model.load_state_dict(torch.load(path))
        preds=predict(model,test_loader)
        all_preds.append(preds)
    
    p=np.array(all_preds).T
    preds=np.average(p,axis=1)
    test["target"]=preds
    return test


# main

In [12]:
train = get_data()
print(train.tail(10),train.shape)

if CFG._train:
    for fold in CFG.fold:
        print(f"fold :{fold}")
        loop_train(train=train,fold=fold,save_model=True)

if CFG._predict:
    test = get_data_test()
    result = loop_predict(test=test)
    result.columns=["comment_id","text","score"]
    result[["comment_id","score"]].to_csv("submission.csv",index=False)
    result.head()

                      id                                       comment_text  \
159561  ffd2e85b07b3c7e4  "\nNo he did not, read it again (I would have ...   
159562  ffd72e9766c09c97  "\n Auto guides and the motoring press are not...   
159563  ffe029a7c79dc7fe  "\nplease identify what part of BLP applies be...   
159564  ffe897e7f7182c90  Catalan independentism is the social movement ...   
159565  ffe8b9316245be30  The numbers in parentheses are the additional ...   
159566  ffe987279560d7ff  ":::::And for the second time of asking, when ...   
159567  ffea4adeee384e90  You should be ashamed of yourself \n\nThat is ...   
159568  ffee36eab5c267c9  Spitzer \n\nUmm, theres no actual article for ...   
159569  fff125370e4aaaf3  And it looks like it was actually you who put ...   
159570  fff46fc426af1f9a  "\nAnd ... I really don't think you understand...   

        toxic  severe_toxic  obscene  threat  insult  identity_hate  target  \
159561      0             0        0       0       

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

KeyboardInterrupt: 